# Correcting an entire video for the fisheye distortion

Fisheye distortion is something we need to fix if we want DeepLabCut to be able to properly learn from our videos. Correcting for this distortion should keep the mice at consistent sizes regardless of their location in the cage.

# 1. Take some pictures

We need to have a set of calibration images so that our program can figure out exactly how the fisheye lens is distorting the picture. Print out the attached calibration image and attach it to a hard flat thing (like a book or something). Move it around and take 20-40 pictures of it with the same camera that the video was recorded with. Drop those files in this folder.

# 2. Calibrate the camera

In this step, we will generate a map of how much each pixel needs to be moved based on the calibration data.

In [1]:
import cv2
import numpy as np
import os
import glob

CHECKERBOARD = (6,9)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('*.jpg')
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
DIM = _img_shape[::-1]

map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, DIM, np.eye(3), balance=0.0), DIM, cv2.CV_16SC2)

Found 18 valid images for calibration


# 3. Undistort your video

Take any videos that you would like to undistort and place them in this folder. Then run the following code. This takes a couple of minutes for an hour of 5fps 1280x960 video, so keep in mind it might take a while. Your video file should show up in this folder as [original name]_undistorted.mp4

In [2]:
def undistort(img):
    return(cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT))
def video(vid_path):
    cap = cv2.VideoCapture(vid_path)
    # fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(vid_path+'_undistorted.mp4', 0x7634706d, 5, DIM)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(0, length):
        ret, frame = cap.read()
        out.write(undistort(frame))
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return
for p in glob.glob('*.mp4'):
    video(p)